In [ ]:
import utils
import pathlib

dataset_name = 'train'

data_folder = pathlib.Path('../data')
json_folder = data_folder / 'json'

xml_file   = data_folder / f'xml/{dataset_name}.xml'
extra_file = json_folder / f'movie_additional.json'
es_folder  = data_folder / 'es'

utils.xml_to_ndjson(xml_file, json_folder, es_folder, extra_file)

Parsing ../data/xml/train.xml file.. 